# Iterating Collections

If we think about iterating over a collection, what we really need is a way to request the **next** item in the collection.

If we can do that, our collection does not require being indexable, nor does it need to be ordered (i.e. we don't need the notion of relative positions of elements in the container).

In [1]:
class Squares:
    def __init__(self, length):
        self._l = length
        self._i = 0
        
    def __len__(self):
        return self._l
    
    def __next__(self):
        if self._i >= self._l:
            raise StopIteration
        else:
            result = self._i**2
            self._i += 1
            return result

In [2]:
s = Squares(10)

In [3]:
# We can't use a for loop
# for i in s:
#     print(i)

In [4]:
for i in range(10):
    print(next(s))

0
1
4
9
16
25
36
49
64
81


# Iterators

There were some downsides that did not resolve (yet!):
* we cannot use a `for` loop
* once we exhaust the iteration (repeatedly calling next), we're essentially done with object. The only way to iterate through it again is to create a new instance of the object.

To resolve these downsides, we create an `iterator` type object.

Iterators are objects that implement:
* a `__next__` method
* an `__iter__` method that simply returns the object itself

In [5]:
class Squares:
    def __init__(self, length):
        self._l = length
        self._i = 0
        
    def __len__(self):
        return self._l
    
    def __next__(self):
        if self._i >= self._l:
            raise StopIteration
        else:
            result = self._i**2
            self._i += 1
            return result
    
    def __iter__(self):
        return self

In [6]:
s = Squares(10)
for i in s:
    print(i)

0
1
4
9
16
25
36
49
64
81


Now we can use for loop to iterate the Squares(10).

In [7]:
for i in s:
    print(i)

But if we run it again, it returns nothing.

The `__iter__` method of an iterator just returns itself.

Once we fully iterate over an iterator, the iterator is **exhausted** and we can no longer use it for iteration purposes.

In [8]:
# We can split the iterator part of our code from the data part.
# And restart the iteration by creating a new iterator every time.
class Squares:
    def __init__(self, length):
        self._l = length
        
    def __len__(self):
        return self._l
    
    def square_n(self, n):
        return n**2
    
class SquaresIterator:
    def __init__(self, squares_obj):
        self._squares_obj = squares_obj
        self._i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._i >= len(self._squares_obj):
            raise StopIteration
        else:
            result = self._squares_obj.square_n(self._i)
            self._i += 1
            return result

s = Squares(10)
s_i = SquaresIterator(s)
for i in s_i:
    print(i)
print('====Recreating an iterator====') 
s_i2 = SquaresIterator(s)
for i in s_i2:
    print(i)

0
1
4
9
16
25
36
49
64
81
====Recreating an iterator====
0
1
4
9
16
25
36
49
64
81


In [9]:
# Combine the two classes.
class Squares:
    def __init__(self, length):
        self._l = length
        
    def __len__(self):
        return self._l
    
    def square_n(self, n):
        return n**2
    
    def __iter__(self):
        return self.SquaresIterator(self)
    
    
    class SquaresIterator:
        def __init__(self, squares_obj):
            self._squares_obj = squares_obj
            self._i = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self._i >= len(self._squares_obj):
                raise StopIteration
            else:
                result = self._squares_obj.square_n(self._i)
                self._i += 1
                return result

s = Squares(10)
for i in s:
    print(i)
print('====Running the second time====') 
for i in s:
    print(i)

0
1
4
9
16
25
36
49
64
81
====Running the second time====
0
1
4
9
16
25
36
49
64
81


It works now that we don't need to create a new iterator for each loop.

## Example: Cyclic Iterators

Iterables do not have to be finite. In fact we can easily create an infinite cyclical iterator.

Here's an example - suppose we have a loop that iterates over some range of integers. As we loop through those integers we want to create a tuple containing the integer and a string that cycles over a finite set (smaller than the list of integers).

```
1, 2, 3, 4, 5, 6, 7, 8, 9, ...

N, S, W, E
```

and we want to generate

```
1N, 2S, 3W, 4E, 5N, 6S, 7W, 8E, 9N, ...
```

In [27]:
class CyclicIterators:
    
    def __init__(self,lst):
        self._lst = lst
        self._i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        result = lst[self._i % len(self._lst)]
        self._i += 1
        return result

In [54]:
lst = ['N','S', 'W', 'E']
n = 20
ci = CyclicIterators(lst)

[f'{i}{next(ci)}' for i in range(1, n+1)]

['1N',
 '2S',
 '3W',
 '4E',
 '5N',
 '6S',
 '7W',
 '8E',
 '9N',
 '10S',
 '11W',
 '12E',
 '13N',
 '14S',
 '15W',
 '16E',
 '17N',
 '18S',
 '19W',
 '20E']

# Iterating Callables

In [56]:
# Define a counter function.
def counter():
    count = 0
    
    def inner():
        nonlocal count
        count += 1
        return count
    
    return inner

In [78]:
class FnIterator:
    
    def __init__(self, fn, stop):
        self._fn = fn
        self._stop = stop
        self.is_consumed = False
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.is_consumed:
            raise StopIteration
        else:
            result = self._fn()
            if result == self._stop:
                self.is_consumed = True
                raise StopIteration
            return result

In [79]:
c = counter()
c_iter = FnIterator(c, 10)

In [80]:
for i in c_iter:
    print(i)

1
2
3
4
5
6
7
8
9


In [85]:
# We can use iter() to create iterator.
c = counter()
c_iter_b = iter(c, 20)

In [86]:
for i in c_iter_b:
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


# Reversed Iteration

For this example we are going to build a custom iterable that returns cards from a 52-card deck.

The deck will be in order of suits (Spades, Hearts, Diamonds and Clubs) and card values (from 2 (lowest) to Ace (highest)).

We are going to use lazy loading - i.e. we are not going to pre-build our card deck.

We just need to recognize that each suit contains `13` cards, so an integer division of the index of the card in the deck will tell us which suit it is. But of course we start indexing at 0.

**Example**

If the requested card is the `6`th in the deck (i.e. index = `5`):

`5 // 13 = 0` ==> first suit (Spades)

If the requested card is the `13`th in the deck (i.e. index = `12`):

`12 // 13 = 0` ==> first suit (Spades)

If the requested card is the `14`th in the deck (i.e. index = `13`):

`13 // 13 = 1` ==> second suit (Hearts)

In [95]:
_SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
_RANKS = tuple(range(2, 11) ) + ('J', 'Q', 'K', 'A')
from collections import namedtuple

Card = namedtuple('Card', 'rank suit')

class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)

    def __len__(self):
        return self.length
    
    def __iter__(self):
        return self.CardDeckIterator(self.length)
        
    def __reversed__(self):
        return self.CardDeckIterator(self.length, reverse=True)
    
    class CardDeckIterator:
        def __init__(self, length, *, reverse=False):
            self.length = length
            self.reverse = reverse
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.i >= self.length:
                raise StopIteration
            else:
                if self.reverse:
                    index = self.length -1 - self.i
                else:
                    index = self.i
                suit = _SUITS[index // len(_RANKS)]
                rank = _RANKS[index % len(_RANKS)]
                self.i += 1
                return Card(rank, suit)

In [96]:
deck = CardDeck()
for card in deck:
    print(card)

Card(rank=2, suit='Spades')
Card(rank=3, suit='Spades')
Card(rank=4, suit='Spades')
Card(rank=5, suit='Spades')
Card(rank=6, suit='Spades')
Card(rank=7, suit='Spades')
Card(rank=8, suit='Spades')
Card(rank=9, suit='Spades')
Card(rank=10, suit='Spades')
Card(rank='J', suit='Spades')
Card(rank='Q', suit='Spades')
Card(rank='K', suit='Spades')
Card(rank='A', suit='Spades')
Card(rank=2, suit='Hearts')
Card(rank=3, suit='Hearts')
Card(rank=4, suit='Hearts')
Card(rank=5, suit='Hearts')
Card(rank=6, suit='Hearts')
Card(rank=7, suit='Hearts')
Card(rank=8, suit='Hearts')
Card(rank=9, suit='Hearts')
Card(rank=10, suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='A', suit='Hearts')
Card(rank=2, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=9, suit='Diamonds')
Card(rank=10, 

In [99]:
for card in reversed(CardDeck()):
    print(card)

Card(rank='A', suit='Clubs')
Card(rank='K', suit='Clubs')
Card(rank='Q', suit='Clubs')
Card(rank='J', suit='Clubs')
Card(rank=10, suit='Clubs')
Card(rank=9, suit='Clubs')
Card(rank=8, suit='Clubs')
Card(rank=7, suit='Clubs')
Card(rank=6, suit='Clubs')
Card(rank=5, suit='Clubs')
Card(rank=4, suit='Clubs')
Card(rank=3, suit='Clubs')
Card(rank=2, suit='Clubs')
Card(rank='A', suit='Diamonds')
Card(rank='K', suit='Diamonds')
Card(rank='Q', suit='Diamonds')
Card(rank='J', suit='Diamonds')
Card(rank=10, suit='Diamonds')
Card(rank=9, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=2, suit='Diamonds')
Card(rank='A', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank=10, suit='Hearts')
Card(rank=9, suit='Hearts')
Card(rank=8, suit='Hearts')
Card(rank=7, suit='Hearts')
Card(ran